In [ ]:
import serial
from time import sleep

ser = serial.Serial()
ser.port = "COM4"
ser.baudrate = 9600
ser.setDTR(False)     #DTRを常にLOWにしReset阻止
ser.timeout = 3
ser.open()

## ライフサイクル
シャッターコントローラー使用時において、各チャネルの制御において2つのモードが存在しており、各サーボは個別に異なるモードを取ることが可能であることを念頭に置く必要がある。
- スイッチ同期モード
- 外部制御モード


シャッターコントローラー基板電源投入時、各サーボはそれぞれスイッチ同期モードである。
PCからの指令によって外部からの介入があると、そのサーボは外部制御モードになる。このとき物理的なスイッチとサーボの状態が異なる状況が発生する。
この状態からスイッチ同期モードに移行するためには、当該サーボのスイッチを操作するか、PCから当該サーボに対してスイッチ同期モードに移行を指示する`set_switch_mode`命令を出すと良い。

## 命令の構文
``
sentence1;sentence2;....\n
``

上のように命令文は`;`で連結できる。ただし連結するメリットもそんなに思いつかないし、命令後読み出しの調整も面倒だ。したがって一つの命令だけでよい!
後ろの`\n`はつけるほうが読み込みが早くなる。そういう仕様である。
```
---example of sentence---
read_switch_mode
open: 2 3
close: 2 3
...etc.
```

## read_switch_mode
スイッチの状態を知らせる。命令後読み出しが必要。

In [ ]:
ser.write(b'read_switch_mode\n')
print(ser.readline().decode().rstrip())

In [ ]:
def read_switch_mode():
    ser.write(b'read_switch_mode\n')
    result_str = ser.readline().decode().rstrip()
    return list(map(int,result_str.split(' ')))

read_switch_mode()

## read_servo_mode
サーボの状態を知らせる。命令後読み出しが必要。

In [ ]:
ser.write(b'read_servo_mode\n')
print(ser.readline().decode().rstrip())

In [ ]:
def read_servo_mode():
    ser.write(b'read_servo_mode\n')
    result_str = ser.readline().decode().rstrip()
    return list(map(int,result_str.split(' ')))

read_servo_mode()

## open
指定された番号のシャッターを開く。命令後読み出しは不要。読みだしたところで読み出せるものはないので、timeoutまで待機することになる。

In [ ]:
ser.write(b'open: 2 3\n') # 0,1,2,3 のうち2,3を開く

In [ ]:
def open(target):
    buffer = 'open:' + ' '.join(list(map(str,target))) + '\n'
    ser.write(buffer.encode())

open([2,3])

## close
指定された番号のシャッターを閉じる。命令後読み出しは不要。読みだしたところで読み出せるものはないので、timeoutまで待機することになる。

In [ ]:
ser.write(b'close: 2 3\n') # 0,1,2,3 のうち2,3を閉じる

In [ ]:
def close(target):
    buffer = 'close:' + ' '.join(list(map(str,target))) + '\n'
    ser.write(buffer.encode())

close([2,3])

## set_to_switch
指定された番号のシャッターを物理的なスイッチの設定と同期させる。これによって当該チャンネルはPC制御モードから外れる。
再び`open` `close`すればPC制御モードになる。

In [ ]:
ser.write(b'set_to_switch: 2 3\n') # 0,1,2,3 のうち2,3を物理スイッチに同期させる

In [ ]:
def set_to_switch(target):
    buffer = 'set_to_switch:' + ' '.join(list(map(str,target))) + '\n'
    ser.write(buffer.encode())
    
set_to_switch([2,3])

## read_serial_num
シリアルナンバーを読み込みます。

In [ ]:
ser.write(b'read_serial_num\n')
print(ser.readline().decode().rstrip())

In [ ]:
def read_serial_num():
    ser.write(b'read_serial_num\n')
    return ser.readline().decode().rstrip()

read_serial_num()

In [ ]:
ser.close()